### PIPELINE

In [68]:
import cv2
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm



# Add the 'src' directory to the Python path
sys.path.append(os.path.join(os.getcwd(), 'src'))
import Evaluator
import Utils



In [69]:


# Define image  names
images = ["0_empty", "0","1","2","3","4","5","6","7","8","9"]


# dictonary for two different sets of parameters
parameters = {
    "over_130": {
        "noise_reduction":  "Gaussian",  
        "filter_size":  3, 
        "clahe":  True, 
        "grid_size":  8,  
        "pct":  50.0,
        "static_foreground_removal":  True,
        "dilation":  True,
        "opening":  False,
        "closing":  True,
        "smoothing_after_morph":  True, 
         
    },
    "below_130": {
        "noise_reduction":  "Median",  
        "filter_size":  3, 
        "clahe":  False, 
        "grid_size":  8,  
        "pct":  35.0,
        "static_foreground_removal":  True,
        "dilation":  True,
        "opening":  True,
        "closing":  True,
        "smoothing_after_morph":  False, 
    }
}







In [70]:

# Initialize the DataFrame with specific column types
df_eval = pd.DataFrame({
    'image': pd.Series(dtype='str'),
    'objects_before_post_processing': pd.Series(dtype='int'),
    'objects_after_post_processing': pd.Series(dtype='int'),
    'TP': pd.Series(dtype='int'),
    'FP': pd.Series(dtype='int'),
    'FN': pd.Series(dtype='int'),
    'Precision': pd.Series(dtype='float'),
    'Recall': pd.Series(dtype='float'),
    'F1': pd.Series(dtype='float'),
    'RMSE': pd.Series(dtype='float'),
    'avg_intensity': pd.Series(dtype='float')
})

In [71]:
# Function to process each parameter combination
def process_image(image):
    
    img_gray = cv2.imread(f"data/images/{image}.jpg", cv2.IMREAD_GRAYSCALE)
    avg_intensity = np.mean(img_gray)
    # Load the image
    img = cv2.imread(f"data/images/{image}.jpg")
    
    

    # Load labels
    labels = pd.read_csv(f"data/labels/labels_{image}.csv")
    labels.columns = ['Label', 'X', 'Y', 'Image', 'Width', 'Height']
    actual_persons = len(labels)
    labels_np = labels[['X', 'Y']].to_numpy()

    # Calulate avg intensity value
    avg_intensity = np.mean(img_gray)

    # if the avg intensity is above 130, use the set of parameters for images with high intensity

    # Select parameters based on intensity
    if avg_intensity >= 130:
        params_set = "over_130"
    else:
        params_set = "below_130"

    # Extract parameters
    noise = parameters[params_set]["noise_reduction"]
    size = parameters[params_set]["filter_size"]
    clahe = parameters[params_set]["clahe"]
    grid = parameters[params_set]["grid_size"]
    p = parameters[params_set]["pct"]
    static_removal = parameters[params_set]["static_foreground_removal"]
    d = parameters[params_set]["dilation"]
    o = parameters[params_set]["opening"]
    c = parameters[params_set]["closing"]
    smooth = parameters[params_set]["smoothing_after_morph"]


    # Preprocess the image
    image_preprocessed = Utils.preprocess_image(img, noise, size, clahe, grid)

    # Background subtraction
    image_no_background = Utils.background_subtraction(image_preprocessed, p, static_removal)

    # Morphological operations
    image_morph = Utils.apply_morphological_operations(image_no_background, d, o, c, smooth)

    # Segment objects with connected components
    num_labels, labels_im, stats, centroids = cv2.connectedComponentsWithStats(image_morph, connectivity=8)
    num_objects_before_post_processing = num_labels - 1

    # Post processing
    bounding_boxes = Utils.post_process_image(image_morph, stats, centroids, nms_threshold=0.5)

    # Evaluate the results
    eval = Evaluator.Evaluator()
    eval.evaluate_bounding_boxes(bounding_boxes, labels_np)

    # get bounding boxes
    tp_boxes, tp_points, fp_boxes, fn_points = eval.get_bounding_boxes()

    # draw result
    #Utils.draw_result(img, tp_boxes, tp_points, fp_boxes, fn_points)
    output_path = f"results/images/{image}_result.jpg"
    img= Utils.draw_result(img, tp_boxes, tp_points, fp_boxes, fn_points)
    # save the result image as rgb
    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Prepare row data
    row = {
        'Image': image, 
        'TP': eval.TP, 
        'FP': eval.FP, 
        'FN': eval.FN, 
        'Precision': eval.get_precision(), 
        'Recall': eval.get_recall(), 
        'F1': eval.get_f1_score(), 
        'RMSE': eval.get_RMSE(actual_persons, eval.TP + eval.FP),
        'Accuracy': eval.get_accuracy(),
        'avg_intensity': avg_intensity
    }
    
    return row

In [72]:

all_results = []
# Initialize the progress bar with the total number of parameter combinations
with tqdm(total=len(images), desc="Progress") as pbar:
    # Iterate over each image and process combinations sequentially
    for image in images:

        result = process_image(image)
                
        if result:  
            all_results.append(result)
            
            
        pbar.update(1)  
all_results = pd.DataFrame(all_results)
   

Progress:   0%|          | 0/11 [00:00<?, ?it/s]

Progress: 100%|██████████| 11/11 [00:37<00:00,  3.41s/it]


In [73]:
all_results

,Image,TP,FP,FN,Precision,Recall,F1,RMSE,Accuracy,avg_intensity
0,0_empty,0,4,0,0.000000,1.000000,0.000000,4.0,0.000000,120.232539
1,0,1,3,0,0.250000,1.000000,0.400000,3.0,25.000000,120.223127
2,1,11,12,11,0.478261,0.500000,0.488889,1.0,47.826087,125.890823
3,2,28,13,23,0.682927,0.549020,0.608696,10.0,68.292683,121.406158
4,3,99,62,35,0.614907,0.738806,0.671186,27.0,61.490683,131.755447
5,4,105,91,30,0.535714,0.777778,0.634441,61.0,53.571429,133.661884
6,5,127,74,44,0.631841,0.742690,0.682796,30.0,63.184080,130.024357
7,6,142,64,39,0.689320,0.784530,0.733850,25.0,68.932039,130.907873
8,7,152,31,81,0.830601,0.652361,0.730769,50.0,83.060109,124.010992
9,8,182,27,63,0.870813,0.742857,0.801762,36.0,87.081340,120.532937


In [74]:
# add column as general parameter set
all_results['parameter_set'] = all_results['avg_intensity'].apply(lambda x: 'over_130' if x >= 130 else 'below_130')

In [75]:
all_results

,Image,TP,FP,FN,Precision,Recall,F1,RMSE,Accuracy,avg_intensity,parameter_set
0,0_empty,0,4,0,0.000000,1.000000,0.000000,4.0,0.000000,120.232539,below_130
1,0,1,3,0,0.250000,1.000000,0.400000,3.0,25.000000,120.223127,below_130
2,1,11,12,11,0.478261,0.500000,0.488889,1.0,47.826087,125.890823,below_130
3,2,28,13,23,0.682927,0.549020,0.608696,10.0,68.292683,121.406158,below_130
4,3,99,62,35,0.614907,0.738806,0.671186,27.0,61.490683,131.755447,over_130
5,4,105,91,30,0.535714,0.777778,0.634441,61.0,53.571429,133.661884,over_130
6,5,127,74,44,0.631841,0.742690,0.682796,30.0,63.184080,130.024357,over_130
7,6,142,64,39,0.689320,0.784530,0.733850,25.0,68.932039,130.907873,over_130
8,7,152,31,81,0.830601,0.652361,0.730769,50.0,83.060109,124.010992,below_130
9,8,182,27,63,0.870813,0.742857,0.801762,36.0,87.081340,120.532937,below_130


In [ ]:
best_params = []
images=["0_empty","0","1","2","3","4","5","6","7","8","9"]
for image in images:
    df_eval = pd.read_csv("results/grid_search/results_" + image + ".csv")
    best_results = df_eval.sort_values(by='F1', ascending=False).head(1)
    # add rank column
    best_results['Rank'] = range(1, len(best_results) + 1)
    best_params.append(best_results)

In [77]:
df_best_params = pd.concat(best_params)
df_best_params

,Image,Noise Reduction,Filter Size,CLAHE,Grid Size,Pct,static_foreground_removal,Dilation,Opening,Closing,...,objects_before_post_processing,objects_after_post_processing,TP,FP,FN,Precision,Recall,F1,RMSE,Rank
264,1,Median,3,False,8,35.0,True,True,False,False,...,3193,29,15,15,7,0.500000,0.681818,0.576923,8.0,1
43,2,Gaussian,3,True,8,50.0,True,True,False,True,...,572,57,44,25,7,0.637681,0.862745,0.733333,18.0,1
351,3,Median,5,False,8,20.0,True,True,True,True,...,161,86,96,35,38,0.732824,0.716418,0.724528,3.0,1
235,4,Median,3,True,8,50.0,True,True,False,True,...,285,144,108,80,27,0.574468,0.800000,0.668731,53.0,1
267,5,Median,3,False,8,35.0,True,True,False,True,...,324,138,123,58,48,0.679558,0.719298,0.698864,10.0,1
43,6,Gaussian,3,True,8,50.0,True,True,False,True,...,318,142,142,64,39,0.689320,0.784530,0.733850,25.0,1
127,7,Gaussian,5,True,8,35.0,True,True,True,True,...,219,107,164,40,69,0.803922,0.703863,0.750572,29.0,1
74,8,Gaussian,3,False,8,35.0,True,True,False,True,...,329,112,181,23,64,0.887255,0.738776,0.806236,41.0,1
234,9,Median,3,True,8,50.0,True,True,False,True,...,499,108,175,28,60,0.862069,0.744681,0.799087,32.0,1


In [78]:
# Ensure the Image column is of the same type in both DataFrames
df_best_params['Image'] = df_best_params['Image'].astype(str)
all_results['Image'] = all_results['Image'].astype(str)

# Ensure avg_intensity is populated in df_best_params
df_best_params = pd.merge(
    df_best_params,
    all_results[['Image', 'avg_intensity']],
    on='Image',
    how='left'
)

# Add the parameter_set column with "best_params"
df_best_params['parameter_set'] = 'best_params'

# Retain only the required columns
df_best_params = df_best_params[['Image', 'TP', 'FP','FN','Precision', 'Recall', 'F1', 'RMSE', 'avg_intensity', 'parameter_set']]
# if acccuracy is NaN, calculate it
df_best_params['Accuracy'] = df_best_params['TP'] / (df_best_params['TP'] + df_best_params['FP'] + df_best_params['FN'])

# Append df_best_params to all_results
all_results = pd.concat([all_results, df_best_params], ignore_index=True)

# Display the resulting DataFrame
all_results

,Image,TP,FP,FN,Precision,Recall,F1,RMSE,Accuracy,avg_intensity,parameter_set
0,0_empty,0,4,0,0.000000,1.000000,0.000000,4.0,0.000000,120.232539,below_130
1,0,1,3,0,0.250000,1.000000,0.400000,3.0,25.000000,120.223127,below_130
2,1,11,12,11,0.478261,0.500000,0.488889,1.0,47.826087,125.890823,below_130
3,2,28,13,23,0.682927,0.549020,0.608696,10.0,68.292683,121.406158,below_130
4,3,99,62,35,0.614907,0.738806,0.671186,27.0,61.490683,131.755447,over_130
5,4,105,91,30,0.535714,0.777778,0.634441,61.0,53.571429,133.661884,over_130
6,5,127,74,44,0.631841,0.742690,0.682796,30.0,63.184080,130.024357,over_130
7,6,142,64,39,0.689320,0.784530,0.733850,25.0,68.932039,130.907873,over_130
8,7,152,31,81,0.830601,0.652361,0.730769,50.0,83.060109,124.010992,below_130
9,8,182,27,63,0.870813,0.742857,0.801762,36.0,87.081340,120.532937,below_130
